In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential




def calculate_and_log_training_time(modelname, start_time, end_time):
    training_duration = end_time - start_time
    # Print training duration in a human-readable format
    hours, rem = divmod(training_duration, 3600)
    minutes, seconds = divmod(rem, 60)
    
    filepath = modelname+"trainingtime.txt"
    with open(filepath, "w") as f:
        f.write(f"Training took {int(hours):02d}:{int(minutes):02d}:{seconds:02f} (hh:mm:ss).")



## runtime
def save_model_config_with_optimizer(model, modelname):
    filepath = modelname+"model_config.txt"
    with open(filepath, 'w') as f:
        # Save layer configurations
        for layer in model.layers:
            f.write(f"Layer: {layer.name}\n")
            f.write(f"Config: {layer.get_config()}\n\n")
        
        # Save optimizer configuration
        optimizer_config = model.optimizer.get_config()
        f.write("Optimizer Config:\n")
        f.write(str(optimizer_config))


def save_model(model, modelname):
    model.save(modelname+".keras")

def save_performance_metrics(history, modelname):
    filepath = modelname+'performance_metrics.csv'
    pd.DataFrame(history.history).to_csv(filepath)
    
    
        
def plot_loss_and_metric(history, metric_name='accuracy', model_name='model'):
    """
    Plots the training and validation loss, and a performance metric from the training history, and saves the plots
    with the model name as a prefix. This version explicitly uses Figures and Axes for better control.

    Parameters:
    - history: Return value from model.fit().
    - metric_name: Name of the performance metric to plot (e.g., 'accuracy', 'mae').
    - model_name: Name of the model, used as a prefix for saving plot images.
    """

    # First plot: Training & validation loss values
    fig, ax = plt.subplots(figsize=(14, 6))  # Explicitly create a Figure and Axes
    ax.plot(history.history['loss'], label='Train Loss')
    ax.plot(history.history['val_loss'], label='Validation Loss')
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper right')
    ax.grid(True)  # Adding grid explicitly
    loss_plot_filename = f'{model_name}_loss_plot.jpg'
    fig.savefig(loss_plot_filename)
    print(f"Loss plot saved as: {loss_plot_filename}")
    plt.close(fig)  # Close the figure

    # Second plot: Training & validation performance metric
    fig, ax = plt.subplots(figsize=(14, 6))  # Again, explicitly create a Figure and Axes for the metric plot
    ax.plot(history.history[metric_name], label=f'Train {metric_name.capitalize()}')
    ax.plot(history.history[f'val_{metric_name}'], label=f'Validation {metric_name.capitalize()}')
    ax.set_title(f'Model {metric_name.capitalize()}')
    ax.set_ylabel(metric_name.capitalize())
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper left')
    ax.grid(True)  # Adding grid explicitly
    metric_plot_filename = f'{model_name}_{metric_name}_plot.jpg'
    fig.savefig(metric_plot_filename)
    print(f"Metric plot saved as: {metric_plot_filename}")
    plt.close(fig)  # Close the figure to free up memory


def save_model_and_config_and_metrics(model, history, modelname = "model"):
    save_model_config_with_optimizer(model, modelname = modelname)
    save_model(model, modelname = modelname)
    save_performance_metrics(history, modelname = modelname)
    plot_loss_and_metric(history, metric_name='mae', model_name=modelname)

In [ ]:
import os

def calculate_and_log_training_time(modelname, start_time, end_time):
    training_duration = end_time - start_time
    hours, rem = divmod(training_duration, 3600)
    minutes, seconds = divmod(rem, 60)
    
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, "trainingtime.txt")
    with open(filepath, "w") as f:
        f.write(f"Training took {int(hours):02d}:{int(minutes):02d}:{seconds:02f} (hh:mm:ss).")

def save_model_config_with_optimizer(model, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, "model_config.txt")
    with open(filepath, 'w') as f:
        for layer in model.layers:
            f.write(f"Layer: {layer.name}\n")
            f.write(f"Config: {layer.get_config()}\n\n")
        
        optimizer_config = model.optimizer.get_config()
        f.write("Optimizer Config:\n")
        f.write(str(optimizer_config))

def save_model(model, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    model_path = os.path.join(model_dir, modelname + ".keras")
    model.save(model_path)

def save_performance_metrics(history, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, 'performance_metrics.csv')
    pd.DataFrame(history.history).to_csv(filepath)

def plot_loss_and_metric(history, metric_name='accuracy', model_name='model'):
    model_dir = model_name
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # First plot: Training & validation loss
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(history.history['loss'], label='Train Loss')
    ax.plot(history.history['val_loss'], label='Validation Loss')
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper right')
    ax.grid(True)
    loss_plot_filename = os.path.join(model_dir, f'{model_name}_loss_plot.jpg')
    fig.savefig(loss_plot_filename)
    print(f"Loss plot saved as: {loss_plot_filename}")
    plt.close(fig)
    
    # Second plot: Training & validation metric
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(history.history[metric_name], label=f'Train {metric_name.capitalize()}')
    ax.plot(history.history[f'val_{metric_name}'], label=f'Validation {metric_name.capitalize()}')
    ax.set_title(f'Model {metric_name.capitalize()}')
    ax.set_ylabel(metric_name.capitalize())
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper left')
    ax.grid(True)
    metric_plot_filename = os.path.join(model_dir, f'{model_name}_{metric_name}_plot.jpg')
    fig.savefig(metric_plot_filename)
    print(f"Metric plot saved as: {metric_plot_filename}")
    plt.close(fig)

def save_model_and_config_and_metrics(model, history, modelname = "model"):
    save_model_config_with_optimizer(model, modelname = modelname)
    save_model(model, modelname = modelname)
    save_performance_metrics(history, modelname = modelname)
    plot_loss_and_metric(history, metric_name='mae', model_name= modelname)

### RGB DataImport

In [2]:
path = "C:/Users/marij/Documents/Universiteit_local/Master_Year1/DeepLearning/Part1_Processed_RGB.pkl"
# Open the pickle file in binary mode
with open(path, 'rb') as file:
    # Load the content of the file into a variable
    RGB_data = pickle.load(file)
    
sampled_RGB_data = RGB_data.sample(n=1000, random_state = 2001)
y = sampled_RGB_data['Age'].values
X = sampled_RGB_data['Image'].values
X = np.stack(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
X_train

array([[[[0.03921569, 0.0627451 , 0.08235294],
         [0.03529412, 0.05882353, 0.07843138],
         [0.03137255, 0.05490196, 0.08235294],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.03529412, 0.05882353, 0.07843138],
         [0.03529412, 0.05882353, 0.07843138],
         [0.03137255, 0.05490196, 0.08235294],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.03137255, 0.05490196, 0.07450981],
         [0.03137255, 0.05490196, 0.07450981],
         [0.02745098, 0.05098039, 0.07843138],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        ...,

        [[0.88235295, 0.827451  , 0.87058824],
         [0.8901961 , 0.8352941 , 0.8784314 ]

In [4]:
X_train.shape

(800, 256, 256, 3)

### Model 1

In [7]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()
calculate_and_log_training_time(modelname = "RBGCNN1", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "RBGCNN1")


c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 986ms/step - loss: 2238.5139 - mae: 35.0162 - val_loss: 733.1816 - val_mae: 22.7503
Epoch 2/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - loss: 745.8562 - mae: 23.4373 - val_loss: 663.2516 - val_mae: 21.8207
Epoch 3/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 732.8185 - mae: 22.9261 - val_loss: 585.2518 - val_mae: 20.5020
Epoch 4/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 928ms/step - loss: 717.8238 - mae: 22.7714 - val_loss: 573.5791 - val_mae: 20.2902
Epoch 5/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 910ms/step - loss: 650.2407 - mae: 21.5747 - val_loss: 560.0024 - val_mae: 19.5840
Epoch 6/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 929ms/step - loss: 593.8788 - mae: 20.4518 - val_loss: 495.6441 - val_mae: 18.9055
Epoch 7/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 576.7543 - mae: 19.6104 - val_loss: 460.8086 - val_mae: 18.3042
Epoch 8/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - loss: 525.4211 - mae: 18.8975 - val_loss: 364.8887 - val_mae: 15.4162
Epoch 9/12


### Model 2

In [8]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()

#CHANGE THIS EACH MODEL
calculate_and_log_training_time(modelname = "RBGCNN2", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "RBGCNN2")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 949ms/step - loss: 1014.7236 - mae: 26.0396 - val_loss: 645.1893 - val_mae: 21.4743
Epoch 2/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 978ms/step - loss: 770.4338 - mae: 23.4671 - val_loss: 627.7648 - val_mae: 21.2076
Epoch 3/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 690.6492 - mae: 22.6613 - val_loss: 494.6158 - val_mae: 18.8132
Epoch 4/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 636.3248 - mae: 20.5852 - val_loss: 562.5933 - val_mae: 20.3129
Epoch 5/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 954ms/step - loss: 512.4792 - mae: 18.7822 - val_loss: 389.7961 - val_mae: 16.2103
Epoch 6/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - loss: 434.7120 - mae: 16.4951 - val_loss: 342.9782 - val_mae: 14.6025
Epoch 7/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 317.7123 - mae: 13.6614 - val_loss: 353.1662 - val_mae: 14.7465
Epoch 8/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 987ms/step - loss: 250.9525 - mae: 11.9901 - val_loss: 366.8786 - val_mae: 14.2939
Epoch 9/12


### Model 3

In [9]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()

#CHANGE THIS EACH MODEL
calculate_and_log_training_time(modelname = "RBGCNN3", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "RBGCNN3")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 951.5848 - mae: 25.4581 - val_loss: 575.1730 - val_mae: 20.2758
Epoch 2/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 931ms/step - loss: 693.9902 - mae: 22.2039 - val_loss: 500.1079 - val_mae: 18.6416
Epoch 3/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 566.1277 - mae: 19.3682 - val_loss: 524.0504 - val_mae: 19.2451
Epoch 4/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 583.6495 - mae: 18.9100 - val_loss: 405.6701 - val_mae: 16.6503
Epoch 5/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 945ms/step - loss: 364.4417 - mae: 14.9667 - val_loss: 422.6143 - val_mae: 16.4496
Epoch 6/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 287.9714 - mae: 13.1479 - val_loss: 341.7905 - val_mae: 14.2121
Epoch 7/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 944ms/step - loss: 268.8148 - mae: 12.1881 - val_loss: 371.3958 - val_mae: 15.1594
Epoch 8/12
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 939ms/step - loss: 192.9655 - mae: 10.6621 - val_loss: 333.3112 - val_mae: 14.0301
Epoch 9/12
2

### Increase sample size by 1000
### Model 1

In [10]:
sampled_RGB_data = RGB_data.sample(n=2000, random_state = 2001)
y = sampled_RGB_data['Age'].values
X = sampled_RGB_data['Image'].values
X = np.stack(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()
calculate_and_log_training_time(modelname = "2000RBGCNN1", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "2000RBGCNN1")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - loss: 1352.4658 - mae: 27.8576 - val_loss: 681.5572 - val_mae: 20.7425
Epoch 2/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 48s 957ms/step - loss: 664.1737 - mae: 21.5744 - val_loss: 605.3840 - val_mae: 20.1337
Epoch 3/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - loss: 539.0345 - mae: 19.3188 - val_loss: 452.4469 - val_mae: 16.9213
Epoch 4/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - loss: 419.8944 - mae: 16.2456 - val_loss: 412.6884 - val_mae: 15.5035
Epoch 5/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 46s 923ms/step - loss: 344.9446 - mae: 14.2759 - val_loss: 388.0883 - val_mae: 14.4851
Epoch 6/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - loss: 277.7482 - mae: 12.3376 - val_loss: 420.1651 - val_mae: 14.2866
Epoch 7/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 224.2502 - mae: 10.9087 - val_loss: 390.4711 - val_mae: 15.0717
Epoch 8/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 80s 986ms/step - loss: 214.0329 - mae: 11.0337 - val_loss: 371.7104 - val_mae: 13.2729
Epoch 9/12
50/

### Model 2

In [13]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()

#CHANGE THIS EACH MODEL
calculate_and_log_training_time(modelname = "2000RBGCNN2", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "2000RBGCNN2")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - loss: 892.3332 - mae: 24.1551 - val_loss: 635.8647 - val_mae: 20.1902
Epoch 2/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - loss: 605.3712 - mae: 20.4546 - val_loss: 564.1740 - val_mae: 18.0135
Epoch 3/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - loss: 483.4496 - mae: 17.7392 - val_loss: 462.6607 - val_mae: 16.9905
Epoch 4/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - loss: 392.3832 - mae: 15.6108 - val_loss: 469.8961 - val_mae: 14.7505
Epoch 5/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 49s 979ms/step - loss: 338.9619 - mae: 13.5303 - val_loss: 411.6151 - val_mae: 13.5428
Epoch 6/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 49s 973ms/step - loss: 279.9233 - mae: 12.3027 - val_loss: 357.3709 - val_mae: 13.2459
Epoch 7/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 940ms/step - loss: 218.8492 - mae: 10.9469 - val_loss: 460.6498 - val_mae: 14.0682
Epoch 8/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 943ms/step - loss: 227.8076 - mae: 10.8165 - val_loss: 351.0320 - val_mae: 12.7723
Epoch 9/12
5

### Model 3

In [14]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()

#CHANGE THIS EACH MODEL
calculate_and_log_training_time(modelname = "2000RBGCNN3", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "2000RBGCNN3")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 49s 953ms/step - loss: 835.8069 - mae: 23.2301 - val_loss: 716.6864 - val_mae: 22.9279
Epoch 2/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 934ms/step - loss: 666.2056 - mae: 20.8799 - val_loss: 743.5247 - val_mae: 23.9762
Epoch 3/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 46s 917ms/step - loss: 592.9092 - mae: 19.6559 - val_loss: 492.4164 - val_mae: 16.4726
Epoch 4/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 48s 955ms/step - loss: 416.6466 - mae: 15.7377 - val_loss: 437.9920 - val_mae: 17.0020
Epoch 5/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 938ms/step - loss: 364.8119 - mae: 14.5042 - val_loss: 401.2849 - val_mae: 13.8667
Epoch 6/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 48s 950ms/step - loss: 308.6489 - mae: 12.8515 - val_loss: 407.8839 - val_mae: 13.6539
Epoch 7/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 933ms/step - loss: 247.1481 - mae: 11.1844 - val_loss: 415.5869 - val_mae: 15.7792
Epoch 8/12
50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 931ms/step - loss: 193.5813 - mae: 10.1273 - val_loss: 367.3989 - val_mae: 13.3972


## Increase the sample size to 4000

In [15]:
sampled_RGB_data = RGB_data.sample(n=4000, random_state = 2001)
y = sampled_RGB_data['Age'].values
X = sampled_RGB_data['Image'].values
X = np.stack(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model 1

In [16]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()
calculate_and_log_training_time(modelname = "4000RBGCNN1", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "4000RBGCNN1")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 989ms/step - loss: 991.3536 - mae: 24.6667 - val_loss: 514.7605 - val_mae: 18.9749
Epoch 2/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 95s 946ms/step - loss: 496.5368 - mae: 17.8858 - val_loss: 414.2482 - val_mae: 15.1277
Epoch 3/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 94s 944ms/step - loss: 390.2432 - mae: 15.1843 - val_loss: 524.0635 - val_mae: 16.3541
Epoch 4/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 94s 936ms/step - loss: 323.8169 - mae: 13.5245 - val_loss: 370.1380 - val_mae: 14.7197
Epoch 5/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 94s 935ms/step - loss: 273.8566 - mae: 12.3434 - val_loss: 370.2566 - val_mae: 13.6792
Epoch 6/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 90s 903ms/step - loss: 216.5243 - mae: 10.9488 - val_loss: 380.5861 - val_mae: 14.7274
Epoch 7/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 91s 915ms/step - loss: 179.1146 - mae: 9.8772 - val_loss: 366.0035 - val_mae: 14.1919
Epoch 8/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 92s 918ms/step - loss: 160.2883 - mae: 9.1705 - val_loss: 362.2559 - va

In [17]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()

#CHANGE THIS EACH MODEL
calculate_and_log_training_time(modelname = "4000RBGCNN2", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "4000RBGCNN2")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 100s 977ms/step - loss: 732.6415 - mae: 21.8682 - val_loss: 527.8630 - val_mae: 17.8196
Epoch 2/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 94s 936ms/step - loss: 459.4997 - mae: 16.5905 - val_loss: 403.9205 - val_mae: 15.5603
Epoch 3/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 94s 936ms/step - loss: 359.0526 - mae: 14.3414 - val_loss: 397.2345 - val_mae: 14.2997
Epoch 4/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 95s 950ms/step - loss: 290.7215 - mae: 12.6314 - val_loss: 359.8949 - val_mae: 14.0218
Epoch 5/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 93s 929ms/step - loss: 247.8128 - mae: 11.4180 - val_loss: 348.6389 - val_mae: 13.9131
Epoch 6/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 93s 926ms/step - loss: 218.7290 - mae: 10.8563 - val_loss: 349.0015 - val_mae: 12.8452
Epoch 7/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 93s 934ms/step - loss: 174.4243 - mae: 9.5561 - val_loss: 332.2126 - val_mae: 13.4407
Epoch 8/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 93s 931ms/step - loss: 140.1494 - mae: 8.4458 - val_loss: 382.3600 - va

## Model 3

In [18]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()

#CHANGE THIS EACH MODEL
calculate_and_log_training_time(modelname = "4000RBGCNN3", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "4000RBGCNN3")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 106s 998ms/step - loss: 766.5151 - mae: 22.2644 - val_loss: 578.5801 - val_mae: 20.5053
Epoch 2/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 485.5310 - mae: 17.4619 - val_loss: 532.5429 - val_mae: 18.9640
Epoch 3/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - loss: 397.8521 - mae: 15.2034 - val_loss: 398.5607 - val_mae: 13.6079
Epoch 4/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - loss: 308.1802 - mae: 12.5628 - val_loss: 377.7584 - val_mae: 14.2541
Epoch 5/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - loss: 233.3449 - mae: 11.2184 - val_loss: 369.1147 - val_mae: 13.0186
Epoch 6/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - loss: 224.4783 - mae: 10.5997 - val_loss: 384.3371 - val_mae: 14.2378
Epoch 7/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - loss: 162.5322 - mae: 9.0716 - val_loss: 343.0667 - val_mae: 12.6385
Epoch 8/12
100/100 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - loss: 119.7447 - mae: 7.5574 - val_loss: 333.5570 - val_mae: 12.6378

## Increase the sample size to 8000
Does not work on my laptop

In [19]:
sampled_RGB_data = RGB_data.sample(n=8000, random_state = 2001)
y = sampled_RGB_data['Age'].values
X = sampled_RGB_data['Image'].values
X = np.stack(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model 1

In [20]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),  # Define the input shape her
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Capture start time
start_time = time.time()
# fit the model
history = model.fit(X, y, epochs=12, validation_split=0.2)
# Capture end time and calculate duration
end_time = time.time()
calculate_and_log_training_time(modelname = "8000RBGCNN1", start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = "8000RBGCNN1")

c:\users\marij\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


KeyboardInterrupt: 